数据透视表与交叉表

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_note_interactivity = "all"
%matplotlib inline


import os
import sys
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import matplotlib.pyplot as plt
import seaborn as sns

1. 数据透视表

数据透视表是根据一个或多个键聚合成一张表的数据，将数据在矩形格式中排列，其中一部分分组键沿行排列（分级索引），另一些是沿着列的（列、数据项）。

Python中的pandas透视表是通过08节的groupby工具和使用分层索引的重塑才做实现的。

dataframe有一个pivot_table方法，并且还有一个顶层的pandas.pivot_table函数

In [11]:
tips = pd.read_csv('code/examples/tips.csv')
tips['tip_pct'] = tips['tip']/tips['total_bill']
tips.pivot_table(index=['day', 'smoker'])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [12]:
#在tip_pct和size上进行聚合，并根据time分组
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'], columns ='smoker')

tip_pct                size          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   0.139622  0.165347  2.000000  2.222222
       Sat   0.158048  0.147906  2.555556  2.476190
       Sun   0.160113  0.187250  2.929825  2.578947
       Thur  0.159744       NaN  2.000000       NaN
Lunch  Fri   0.187735  0.188937  3.000000  1.833333
       Thur  0.160311  0.163863  2.500000  2.352941

In [14]:
#增加透视表的均值项margins=True
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'], columns='smoker', margins=True)

tip_pct                          size                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   0.139622  0.165347  0.158916  2.000000  2.222222  2.166667
       Sat   0.158048  0.147906  0.153152  2.555556  2.476190  2.517241
       Sun   0.160113  0.187250  0.166897  2.929825  2.578947  2.842105
       Thur  0.159744       NaN  0.159744  2.000000       NaN  2.000000
Lunch  Fri   0.187735  0.188937  0.188765  3.000000  1.833333  2.000000
       Thur  0.160311  0.163863  0.161301  2.500000  2.352941  2.459016
All          0.159328  0.163196  0.160803  2.668874  2.408602  2.569672

In [15]:
#当使用不同的聚合函数时，将函数传递给aggfunc参数。
tips.pivot_table('tip_pct', index=['time', 'smoker'], columns='day',aggfunc=len, margins=True)

day             Fri   Sat   Sun  Thur    All
time   smoker                               
Dinner No       3.0  45.0  57.0   1.0  106.0
       Yes      9.0  42.0  19.0   NaN   70.0
Lunch  No       1.0   NaN   NaN  44.0   45.0
       Yes      6.0   NaN   NaN  17.0   23.0
All            19.0  87.0  76.0  62.0  244.0

In [16]:
tips.pivot_table('tip_pct', index=['time', 'size', 'smoker'], columns='day', aggfunc='mean', fill_value=0)

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
            Yes     0.000000  0.144995  0.152660  0.000000
       4    No      0.000000  0.150096  0.148143  0.000000
            Yes     0.117750  0.124515  0.193370  0.000000
       5    No      0.000000  0.000000  0.206928  0.000000
            Yes     0.000000  0.106572  0.065660  0.000000
       6    No      0.000000  0.000000  0.103799  0.000000
Lunch  1    No      0.000000  0.000000  0.000000  0.181728
            Yes     0.223776  0.000000  0.000000  0.000000
       2    No      0.000000  0.000000  0.000000  0.166005
            Yes     0.181969  0.000000  0.000000  0.158843
       3    No      0.187735  0.000000  0.000000  0.084246
            Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706

pivot_table方法参数：

values————需要聚合的列名；默认情况下聚合所有数值型的列

index————在结果透视表的行上进行分组的列名或其他分组键

columns————在结果透视表的列上进行分组的列名或其他分组键

aggfunc————聚合函数或函数列表（默认是‘mean’）；可以是groupby上下文的任意有效函数

fill_value————在结果表中替换缺失值的值

dropna————若为True，将不包含所有条目均为NA的列

margins————添加行/列的小计和总计，默认为F

2. 交叉表crosstab（透视表的一个特殊情况，计算的是分组中的频率）

In [18]:
# pd.crosstab(data.Nationality, data.Handedness, margins=True)

In [20]:
#crosstab的前两个参数可以是数组、Series或数组的列表
pd.crosstab([tips.time, tips.day], tips.smoker, margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244